In [42]:
# AWS Account Setup
from config.aws_config import AWSConfig

config = AWSConfig(region="us-west-2")
config.validate_credentials()
print("✓ AWS credentials validated")

✓ AWS credentials validated


In [43]:
#Create IAM Resources
from src.iam_manager import IAMManager

iam_manager = IAMManager(config)

# Create KB execution role
kb_role = iam_manager.create_knowledge_base_execution_role('bedrock-kb-role')
print(f"✓ Created KB role: {kb_role['role_arn']}")

# Create policies
fm_policy = iam_manager.create_foundation_model_policy('bedrock-models-policy')
s3_policy = iam_manager.create_s3_bucket_policy('bedrock-s3-policy', ['my-bedrock-documents'])

# Attach policies
iam_manager.attach_policy_to_role('bedrock-kb-role', fm_policy['policy_arn'])
iam_manager.attach_policy_to_role('bedrock-kb-role', s3_policy['policy_arn'])
print("✓ IAM resources created and configured")


✓ Created KB role: arn:aws:iam::390844762889:role/bedrock-kb-role
✓ IAM resources created and configured


In [44]:
# Create OpenSearch Serverless Resources
from src.oss_security import OSSSecurityManager

oss_manager = OSSSecurityManager(config)

# Create security policies
enc_policy = oss_manager.create_encryption_policy('bedrock-encryption-policy')
net_policy = oss_manager.create_network_policy('bedrock-network-policy', ['bedrock-collection'])
data_policy = oss_manager.create_data_access_policy('bedrock-data-access-policy', ['bedrock-collection'], [kb_role['role_arn']])

print("✓ OpenSearch Serverless policies created")


✓ OpenSearch Serverless policies created


In [45]:
import boto3

bucket_uswest = "my-bedrock-documents-us-west-2-7055703"  # change if needed

s3_usw2 = boto3.client("s3", region_name="us-west-2")

s3_usw2.create_bucket(
    Bucket=bucket_uswest,
    CreateBucketConfiguration={"LocationConstraint": "us-west-2"},
)
# print(f"✓ S3 bucket created: {bucket_uswest['bucket_name']}")
print("Created bucket:", bucket_uswest)

BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [46]:
import boto3

bucket_uswest = "my-bedrock-documents-us-west-2-7055703"
s3_usw2 = boto3.client("s3", region_name="us-west-2")

print(s3_usw2.get_bucket_location(Bucket=bucket_uswest))


{'ResponseMetadata': {'RequestId': 'TNPNSYXE8ZE8D62R', 'HostId': 'SGvzzhfKU/mXMKgMgFiI5MJJL4qv1ymJ+lhrLK9ijZPLhQfGzz8mBDrZqzvNLMr2sjPvMa60rmE=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'SGvzzhfKU/mXMKgMgFiI5MJJL4qv1ymJ+lhrLK9ijZPLhQfGzz8mBDrZqzvNLMr2sjPvMa60rmE=', 'x-amz-request-id': 'TNPNSYXE8ZE8D62R', 'date': 'Mon, 08 Dec 2025 07:26:09 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'LocationConstraint': 'us-west-2'}


In [47]:
import boto3

oss = boto3.client("opensearchserverless")

resp = oss.list_collections(maxResults=50)

for c in resp.get("collectionSummaries", []):
    print("Name:", c["name"], "| ID:", c["id"], "| Status:", c["status"])


Name: bedrock-sample-rag-7095841 | ID: 2fx3rnr6p086xny3c04 | Status: ACTIVE
Name: bedrock-sample-rag-7125611-f | ID: 89bab1jil24609rk7v2d | Status: ACTIVE
Name: bedrock-sample-rag-7122641 | ID: blcuo2bh8pxi0rkf24sc | Status: ACTIVE
Name: bedrock-sample-rag-6140503-f | ID: cr3xdx8komynbuzxd6gb | Status: ACTIVE
Name: bedrock-sample-rag-7055703-h | ID: eaj01t4vjgawaalzhjo1 | Status: ACTIVE
Name: bedrock-sample-rag-7055703-c | ID: hfnwzycxnedbgwb9jt3j | Status: ACTIVE
Name: bedrock-sample-rag-7102501 | ID: nvzucpspi4jrzqzvrsjl | Status: ACTIVE
Name: bedrock-sample-rag-7132337-f | ID: s8qfexfq1wo4pjple9m9 | Status: ACTIVE
Name: bedrock-sample-rag-7055703-s | ID: sscbudrxjxoow16btlfk | Status: ACTIVE
Name: bedrock-sample-rag-7055703-f | ID: ugelnzuw18qccue62ond | Status: ACTIVE


In [48]:
import boto3

for r in ["us-east-1", "us-west-2", "eu-central-1"]:
    try:
        oss = boto3.client("opensearchserverless", region_name=r)
        resp = oss.list_collections(maxResults=50)
        names = [c["name"] for c in resp.get("collectionSummaries", [])]
        print(r, ":", names)
    except Exception as e:
        print(r, "-> error:", e)

us-east-1 : []
us-west-2 : ['bedrock-sample-rag-7095841', 'bedrock-sample-rag-7125611-f', 'bedrock-sample-rag-7122641', 'bedrock-sample-rag-6140503-f', 'bedrock-sample-rag-7055703-h', 'bedrock-sample-rag-7055703-c', 'bedrock-sample-rag-7102501', 'bedrock-sample-rag-7132337-f', 'bedrock-sample-rag-7055703-s', 'bedrock-sample-rag-7055703-f']
eu-central-1 -> error: An error occurred (AccessDeniedException) when calling the ListCollections operation: User: arn:aws:sts::390844762889:assumed-role/bedrock-workshop-studio-SageMakerExecutionRole-sdZJVpsJ7VJd/SageMaker is not authorized to perform: aoss:ListCollections on resource: arn:aws:aoss:eu-central-1:390844762889:collection/* with an explicit deny in a service control policy


In [49]:
from config.aws_config import AWSConfig
from src.vector_store import VectorIndexManager

# Create a fresh AWSConfig pointing to us-west-2
config = AWSConfig(region="us-west-2")   # 👈 IMPORTANT

# Sanity check: what does this config "see"?
oss = config.get_client("opensearchserverless")
print("Region from config:", oss.meta.region_name)

resp = oss.list_collections(maxResults=50)
print("Collections in this region:")
for c in resp.get("collectionSummaries", []):
    print("  -", c["name"])

Region from config: us-west-2
Collections in this region:
  - bedrock-sample-rag-7095841
  - bedrock-sample-rag-7125611-f
  - bedrock-sample-rag-7122641
  - bedrock-sample-rag-6140503-f
  - bedrock-sample-rag-7055703-h
  - bedrock-sample-rag-7055703-c
  - bedrock-sample-rag-7102501
  - bedrock-sample-rag-7132337-f
  - bedrock-sample-rag-7055703-s
  - bedrock-sample-rag-7055703-f


In [50]:
import importlib
import src.vector_store as vector_store

importlib.reload(vector_store)

from src.vector_store import VectorIndexManager

vector_manager = VectorIndexManager(config)

index = vector_manager.create_vector_index(
    collection_name="bedrock-sample-rag-7055703-s",
    index_name="bedrock-vectors-faiss-1024-v2",
    vector_dimension=1024,
    similarity_metric="cosine",
)

print(index)
print(
    f"✓ Vector index created: {index['index_name']} "
    f"in collection {index['collection_name']}"
)

VectorIndexManager region from AWSConfig: us-west-2
oss_client region: us-west-2
Collections visible to VectorIndexManager:
  - bedrock-sample-rag-7095841
  - bedrock-sample-rag-7125611-f
  - bedrock-sample-rag-7122641
  - bedrock-sample-rag-6140503-f
  - bedrock-sample-rag-7055703-h
  - bedrock-sample-rag-7055703-c
  - bedrock-sample-rag-7102501
  - bedrock-sample-rag-7132337-f
  - bedrock-sample-rag-7055703-s
  - bedrock-sample-rag-7055703-f
batch_get_collection response: {'collectionDetails': [{'id': 'sscbudrxjxoow16btlfk', 'name': 'bedrock-sample-rag-7055703-s', 'status': 'ACTIVE', 'type': 'VECTORSEARCH', 'arn': 'arn:aws:aoss:us-west-2:390844762889:collection/sscbudrxjxoow16btlfk', 'kmsKeyArn': 'auto', 'standbyReplicas': 'ENABLED', 'createdDate': 1765088840105, 'lastModifiedDate': 1765088863634, 'collectionEndpoint': 'https://sscbudrxjxoow16btlfk.us-west-2.aoss.amazonaws.com', 'dashboardEndpoint': 'https://sscbudrxjxoow16btlfk.us-west-2.aoss.amazonaws.com/_dashboards'}], 'collectio

In [51]:
import boto3

iam = boto3.client("iam")
for r in iam.list_roles()["Roles"]:
    name = r["RoleName"]
    if "Bedrock" in name or "Knowledge" in name or "KB" in name:
        print(name, r["Arn"])

AmazonBedrockExecutionRoleForKnowledgeBase_6140503-f arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_6140503-f
AmazonBedrockExecutionRoleForKnowledgeBase_7055703-c arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_7055703-c
AmazonBedrockExecutionRoleForKnowledgeBase_7055703-f arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_7055703-f
AmazonBedrockExecutionRoleForKnowledgeBase_7055703-h arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_7055703-h
AmazonBedrockExecutionRoleForKnowledgeBase_7055703-s arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_7055703-s
AmazonBedrockExecutionRoleForKnowledgeBase_7095841 arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_7095841
AmazonBedrockExecutionRoleForKnowledgeBase_7102501 arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_7102501
AmazonBedrockExecutionRoleForKnowledgeBase_712264

In [52]:
# Create Knowledge Base
from src.knowledge_base_manager import BedrockKnowledgeBase

kb_manager = BedrockKnowledgeBase(config)

role_arn = "arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_7055703-s"

vector_store_config = {
    "collectionArn": "arn:aws:aoss:us-west-2:390844762889:collection/sscbudrxjxoow16btlfk",
    "vectorIndexName": "bedrock-vectors-faiss-1024-v2",  # 👈 USE NEW INDEX HERE
    "fieldMapping": {
        "vectorField": "embedding",
        "textField": "content",
        "metadataField": "metadata",
    },
}

kb = kb_manager.create_knowledge_base(
    kb_name="my-knowledge-base-v2",
    kb_description="My RAG knowledge base-mdfixed",
    role_arn=role_arn,
    vector_store_config=vector_store_config,
    embedding_model="amazon.titan-embed-text-v2:0",
    generation_model="anthropic.claude-3-sonnet-20240229-v1:0",
)

print(f"✓ Knowledge base created: {kb['kb_id']}")
kb

✓ Knowledge base created: RWNPUQ7R4Q


{'kb_id': 'RWNPUQ7R4Q',
 'kb_name': 'my-knowledge-base-v2',
 'kb_description': 'My RAG knowledge base-mdfixed',
 'status': 'ACTIVE',
 'role_arn': 'arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_7055703-s',
 'created_at': datetime.datetime(2025, 12, 8, 7, 22, 57, 193875, tzinfo=tzlocal()),
 'updated_at': datetime.datetime(2025, 12, 8, 7, 22, 57, 193875, tzinfo=tzlocal()),
 'failure_reasons': []}

In [55]:
kb_name = "my-knowledge-base-v2"
import time
while True:
    kb = kb_manager._get_knowledge_base_by_name(kb_name)
    print("Status:", kb["status"])
    if kb["status"] == "ACTIVE":
        print("KB READY! KB ID =", kb["kb_id"])
        break
    time.sleep(5)

Status: ACTIVE
KB READY! KB ID = RWNPUQ7R4Q


In [56]:
import inspect
print(inspect.signature(kb_manager.create_data_source))

(kb_id: str, data_source_name: str, data_source_config: Dict[str, Any], data_source_type: str) -> Dict[str, Any]


In [57]:
import importlib
import src.knowledge_base_manager as kb_mod

importlib.reload(kb_mod)
from src.knowledge_base_manager import BedrockKnowledgeBase

kb_manager = BedrockKnowledgeBase(config)

In [58]:
# Create Data Source (whole bucket, no prefixes)
data_source = kb_manager.create_data_source(
    kb["kb_id"],
    "my-s3-source-usw2",
    {
        "bucket_name": bucket_uswest,
        # no inclusion_prefixes → whole bucket, helper will omit inclusionPrefixes
    },
    "S3",
)

print("✓ Data source created:", data_source["data_source_id"])

data_source

✓ Data source created: JCTGAKO1NU


{'data_source_id': 'JCTGAKO1NU',
 'data_source_name': 'my-s3-source-usw2',
 'status': 'AVAILABLE',
 'created_at': datetime.datetime(2025, 12, 8, 7, 27, 28, 784364, tzinfo=tzlocal())}

In [59]:
import boto3
bedrock_agent = config.get_client("bedrock-agent")

resp = bedrock_agent.list_data_sources(knowledgeBaseId=kb["kb_id"])

print("=== Data sources for KB:", kb["kb_id"], "===")
for ds in resp.get("dataSourceSummaries", []):
    print("NAME:", ds["name"], "| ID:", ds["dataSourceId"])

=== Data sources for KB: RWNPUQ7R4Q ===
NAME: my-s3-source-usw2 | ID: JCTGAKO1NU


In [60]:
resp_kb = bedrock_agent.list_knowledge_bases()
for kbinfo in resp_kb.get("knowledgeBaseSummaries", []):
    print("KB:", kbinfo["name"], "| KB_ID:", kbinfo["knowledgeBaseId"])

KB: autogenerated-filters-kb-7073721 | KB_ID: ZZWPDQGJZL
KB: reranking-kb-7122641 | KB_ID: YM9ITZZNKL
KB: bedrock-multi-modal-kb-7125611 | KB_ID: YHWZ55VYOF
KB: metadata-filtering-kb-7094429 | KB_ID: UAHVKKU6ZU
KB: bedrock-sample-knowledge-base-6140503 | KB_ID: TGOPTHYRUB
KB: my-knowledge-base-v2 | KB_ID: RWNPUQ7R4Q
KB: semantic-kb-7055703 | KB_ID: LEFKDNNQHH
KB: standard-kb-7055703 | KB_ID: JIYBZITCJ0
KB: custom-chunking-kb-7055703 | KB_ID: GINZHHLCXE
KB: c90d-cgdemo | KB_ID: FZWPGVY51B
KB: hierarchical-kb-7055703 | KB_ID: FUGISUMFHU
KB: reranking-kb-7102501 | KB_ID: DHF2VECXCD


In [61]:
data_source_id = next(
    ds["dataSourceId"]
    for ds in resp.get("dataSourceSummaries", [])
    if ds["name"] == "my-s3-source-usw2"
)
print("Using existing data source:", data_source_id)
# bucket_uswest = "my-bedrock-documents-us-west-2-7055703"

Using existing data source: JCTGAKO1NU


In [62]:
data_source_id

'JCTGAKO1NU'

In [63]:
import boto3
s3_client = boto3.client("s3")
import os
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("../Assignment/data", "my-bedrock-documents")

uploading file ../Assignment/data/Nestlé-India-AR-2024-25.pdf to my-bedrock-documents
uploading file ../Assignment/data/.ipynb_checkpoints/Nestlé-India-AR-2024-25-checkpoint.pdf to my-bedrock-documents


In [64]:
import os
import boto3

s3_client = boto3.client("s3", region_name="us-west-2")

def upload_directory(path, bucket_name, prefix=""):
    for root, dirs, files in os.walk(path):
        for file in files:
            full_path = os.path.join(root, file)
            relative_path = os.path.relpath(full_path, path)
            s3_key = os.path.join(prefix, relative_path).replace("\\", "/")

            print(f"Uploading {full_path} -> s3://{bucket_name}/{s3_key}")
            s3_client.upload_file(full_path, bucket_name, s3_key)

# Adjust local path if needed
# local_path = "/mnt/efs/user-default-efs/Assignment/data"
upload_directory("../Assignment/data", bucket_uswest)


Uploading ../Assignment/data/Nestlé-India-AR-2024-25.pdf -> s3://my-bedrock-documents-us-west-2-7055703/Nestlé-India-AR-2024-25.pdf
Uploading ../Assignment/data/.ipynb_checkpoints/Nestlé-India-AR-2024-25-checkpoint.pdf -> s3://my-bedrock-documents-us-west-2-7055703/.ipynb_checkpoints/Nestlé-India-AR-2024-25-checkpoint.pdf


In [21]:
response = s3_client.list_objects_v2(Bucket=bucket_uswest)
for obj in response.get("Contents", []):
    print(obj["Key"])

.ipynb_checkpoints/Nestlé-India-AR-2024-25-checkpoint.pdf
Nestlé-India-AR-2024-25.pdf


In [65]:
import boto3

s3 = boto3.client("s3")
loc = s3.get_bucket_location(Bucket=bucket_uswest)
print(loc)

{'ResponseMetadata': {'RequestId': 'ZE55QG8KBT0P9KZM', 'HostId': 'Bw1HAHxxL8+41nnxfV99HWmx95d3GF8nn9Tf0GaJRgkw8VSzVe7bzwMDxvDkxiQtyocoahxi1iY=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Bw1HAHxxL8+41nnxfV99HWmx95d3GF8nn9Tf0GaJRgkw8VSzVe7bzwMDxvDkxiQtyocoahxi1iY=', 'x-amz-request-id': 'ZE55QG8KBT0P9KZM', 'date': 'Mon, 08 Dec 2025 07:28:06 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'LocationConstraint': 'us-west-2'}


In [66]:
#  Start Ingestion
from src.ingestion_manager import IngestionJobManager

ingestion_manager = IngestionJobManager(config)

# Start ingestion
job = ingestion_manager.start_ingestion_job(
    kb_id=kb["kb_id"],
    data_source_id=data_source_id,
)

print("✓ Ingestion started:", job["ingestion_job_id"])

ingestion_manager.wait_for_ingestion_job_complete(
    kb_id=kb["kb_id"],
    data_source_id=data_source_id,
    ingestion_job_id=job["ingestion_job_id"],
    max_wait_seconds=3600
)

print("✓ Ingestion completed")

✓ Ingestion started: BQ2VR5HWCP
✓ Ingestion completed


In [68]:
kb = kb_manager._get_knowledge_base_by_name("my-knowledge-base-v2")
print(kb)

{'kb_id': 'RWNPUQ7R4Q', 'kb_name': 'my-knowledge-base-v2', 'kb_description': 'My RAG knowledge base-mdfixed', 'status': 'ACTIVE', 'role_arn': 'arn:aws:iam::390844762889:role/AmazonBedrockExecutionRoleForKnowledgeBase_7055703-s', 'created_at': datetime.datetime(2025, 12, 8, 7, 22, 57, 193875, tzinfo=tzlocal()), 'updated_at': datetime.datetime(2025, 12, 8, 7, 22, 57, 193875, tzinfo=tzlocal()), 'failure_reasons': []}


In [73]:
kb["kb_id"]

'RWNPUQ7R4Q'

In [87]:
#  Test Retrieval
import importlib
import src.retrieval_api as retrieval_api_mod

importlib.reload(retrieval_api_mod)
from src.retrieval_api import RetrieveAPI
from src.retrieval_config import RetrievalConfiguration, RetrievalType

retrieval_api = RetrieveAPI(config)

retrieval_config = RetrievalConfiguration(
    retrieval_type=RetrievalType.SEMANTIC,
    max_results=5,
)


results = retrieval_api.retrieve(
    kb["kb_id"],                           # knowledge_base_id
    "What are the company revenues?",     # query
    retrieval_config                      # config
)

print(f"✓ Retrieved {len(results)} documents")

for i, r in enumerate(results, start=1):
    print(f"\nResult {i}")
    print("Relevance:", r.relevance_score)
    print("Chunk ID:", r.chunk_id)
    print("Source document:", r.source_document)
    print("Location:", r.location)
    print("Content:", r.content[:400], "...")
# print(f"✓ Retrieved {len(response.results)} documents")


VectorIndexManager region from AWSConfig: us-west-2
oss_client region: us-west-2
✓ Retrieved 5 documents

Result 1
Relevance: 0.6724329
Chunk ID: 1%3A0%3AOWjc_JoBtnpmSH9B_cj8
Source document: s3://my-bedrock-documents-us-west-2-7055703/Nestlé-India-AR-2024-25.pdf
Location: s3://my-bedrock-documents-us-west-2-7055703/Nestlé-India-AR-2024-25.pdf
Content: 7,848.3 200,775.0 9,748.6 242,754.8     24 REVENUE FROM OPERATIONS      a) Sale of products      Revenue from sale of goods is recognised on transfer of control of goods to the buyer. Revenue is measured at the price charged to the customer and are recorded net of returns (if any), trade discounts, rebates, other pricing allowances to trade/consumer, when it is probable that the associated economi ...

Result 2
Relevance: 0.6724329
Chunk ID: 1%3A0%3AzFvc_JoB5qYsXDtg7OSZ
Source document: s3://my-bedrock-documents-us-west-2-7055703/.ipynb_checkpoints/Nestlé-India-AR-2024-25-checkpoint.pdf
Location: s3://my-bedrock-documents-us-west-2-70557

In [97]:
#  Test Retrieve and Generate
from src.retrieve_and_generate_api import RetrieveAndGenerateAPI, GenerationConfig
import importlib
import src.retrieve_and_generate_api as rag_mod
importlib.reload(rag_mod)

rag_api = RetrieveAndGenerateAPI(config)
gen_config = GenerationConfig(max_tokens=512, temperature=0.7)

response = rag_api.retrieve_and_generate(
    kb['kb_id'],
    'What are the company revenues?',
    generation_config=gen_config,
)
print(f"✓ Generated response: {response.generated_text[:500]}...")


VectorIndexManager region from AWSConfig: us-west-2
oss_client region: us-west-2
✓ Generated response: According to the context provided, the company revenues from sale of products are:

For the financial year ended 31st March 2025:
Contracted Price: ₹208,696.7 million
Less: Rebates, discounts, allowances etc.: ₹7,921.7 million
Revenue from Sale of Products: ₹200,775.0 million

Source 1 and Source 2 state:
"(J in million) 
Particulars Financial Year ended 31st March, 2025
Contracted Price 208,696.7
Less: Rebates, discounts, allowances etc. 7,921.7  
Sale of products 200,775.0"

For the financial ...


In [41]:
# Cleanup 
from src.cleanup_manager import ResourceCleanupManager

cleanup_manager = ResourceCleanupManager(config)

# Clean up knowledge base
cleanup_results = cleanup_manager.cleanup_knowledge_base_resources(
    kb_id=kb['kb_id'],
    kb_manager=kb_manager,
    delete_s3_buckets=True,
    delete_iam_roles=True,
    confirm=True
)

# Print report
report = cleanup_manager.generate_cleanup_report(cleanup_results)
print(report)

RESOURCE CLEANUP REPORT

Knowledge Base Cleanup:
  - KB Deleted: True
  - Data Sources Deleted: 2

------------------------------------------------------------
Total Resources Deleted: 3

No errors encountered.
